In [2]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import json

from datetime import datetime, timedelta

In [3]:
# Read bloomberg data
raw_bloomberg_df = pd.read_csv('usd_idr_data.csv', header=1)

# Rename header
raw_bloomberg_df = raw_bloomberg_df.rename(columns={'Unnamed: 0': 'Datetime'})

raw_bloomberg_df

,Datetime,PX_OPEN,PX_LAST,PX_HIGH,PX_LOW,BID,ASK,PX_MID,high,low
0,1991-11-05,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
1,1991-11-06,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
2,1991-11-07,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
3,1991-11-08,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
4,1991-11-11,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
...,...,...,...,...,...,...,...,...,...,...
8379,2024-10-22,15534.0,15560.0,15572.0,15534.0,15555.0,15565.0,15560.0,15572.0,15534.0
8380,2024-10-23,15590.0,15620.0,15633.0,15590.0,15615.0,15625.0,15620.0,15633.0,15590.0
8381,2024-10-24,15640.0,15584.0,15644.0,15580.0,15583.0,15585.0,15584.0,15644.0,15580.0
8382,2024-10-25,15622.0,15640.0,15643.0,15603.0,15635.0,15645.0,15640.0,15643.0,15603.0


In [3]:
# Read bloomberg data
raw_bloomberg_df = pd.read_csv('btc_daily.csv', header=0)

# # Rename header
# raw_bloomberg_df = raw_bloomberg_df.rename(columns={'Unnamed: 0': 'Datetime'})

raw_bloomberg_df

,coin,opentime,openprice,highprice,lowprice,closeprice,volume,closetime,quotevolume,trades,taker_buy_volume,taker_buy_quote,unused
0,BTC,1567900800000,10000.00,10412.65,10000.00,10391.63,3096.291,1567987199999,3.209628e+07,3754,0.039,3.933563e+02,0
1,BTC,1567987200000,10316.62,10475.54,10077.22,10307.00,14824.373,1568073599999,1.524472e+08,11296,5452.451,5.619740e+07,0
2,BTC,1568073600000,10307.00,10382.97,9940.87,10102.02,9068.955,1568159999999,9.271765e+07,10089,6395.462,6.552636e+07,0
3,BTC,1568160000000,10094.27,10293.11,9884.31,10159.55,10897.922,1568246399999,1.097862e+08,13811,7724.442,7.784116e+07,0
4,BTC,1568246400000,10163.06,10450.13,10042.12,10415.13,15609.634,1568332799999,1.594942e+08,20060,10117.798,1.031607e+08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,BTC,1730073600000,67992.70,70331.00,67584.20,69940.00,256128.622,1730159999999,1.764992e+10,3334954,135854.890,9.364715e+09,0
1878,BTC,1730160000000,69939.90,73660.00,69733.00,72739.50,382502.060,1730246399999,2.750227e+10,4796294,199025.299,1.431096e+10,0
1879,BTC,1730246400000,72739.50,72984.90,71444.20,72349.00,227108.808,1730332799999,1.639468e+10,3249471,113211.086,8.173511e+09,0
1880,BTC,1730332800000,72349.00,72720.30,69705.00,70321.90,256485.328,1730419199999,1.826086e+10,3578458,122732.500,8.739091e+09,0


In [75]:
# # Find rows where PX_OPEN, PX_LAST, PX_HIGH, and PX_LOW have the same value
# same_value_rows = raw_bloomberg_df[
#     (raw_bloomberg_df['PX_OPEN'] == raw_bloomberg_df['PX_LAST']) &
#     (raw_bloomberg_df['PX_OPEN'] == raw_bloomberg_df['PX_HIGH']) &
#     (raw_bloomberg_df['PX_OPEN'] == raw_bloomberg_df['PX_LOW'])
# ]

# # Display the rows with the same values
# same_value_rows

In [154]:
# Make sure that 'Datetime; is in the correct format
raw_bloomberg_df['Datetime'] = pd.to_datetime(raw_bloomberg_df['Datetime'])

# Filter the data from year 1995 only because the years before has the same value for the OHLC
df = raw_bloomberg_df[raw_bloomberg_df['Datetime'].dt.year >= 1994]

# Take necessary columns
df = df[['Datetime','PX_OPEN','PX_HIGH','PX_LOW','PX_LAST']]

# Rename the columns
df.columns = ['date','open', 'high', 'low', 'close']

df

,date,open,high,low,close
564,1994-01-03,2109.0,2109.0,2109.0,2109.0
565,1994-01-04,2106.0,2106.0,2106.0,2106.0
566,1994-01-05,2105.0,2105.0,2105.0,2105.0
567,1994-01-06,2103.0,2103.0,2103.0,2103.0
568,1994-01-07,2102.0,2102.0,2102.0,2102.0
...,...,...,...,...,...
8379,2024-10-22,15534.0,15572.0,15534.0,15560.0
8380,2024-10-23,15590.0,15633.0,15590.0,15620.0
8381,2024-10-24,15640.0,15644.0,15580.0,15584.0
8382,2024-10-25,15622.0,15643.0,15603.0,15640.0


In [118]:
# Display rows with any null values in the 'open', 'high', 'low', or 'close' columns
missing_values_df = df[df[['open', 'high', 'low', 'close']].isnull().any(axis=1)]
missing_values_df

,date,open,high,low,close
4210,2008-01-01,NaN,NaN,NaN,NaN
4217,2008-01-10,NaN,NaN,NaN,NaN
4405,2008-10-02,NaN,NaN,NaN,9478.0
4611,2009-07-20,NaN,NaN,NaN,NaN
4958,2010-11-17,8947.0,8947.0,NaN,NaN
4967,2010-11-30,9018.0,9018.0,NaN,NaN
4972,2010-12-07,9013.0,9013.0,NaN,NaN
4985,2010-12-24,9044.0,9044.0,NaN,NaN
5014,2011-02-03,9024.0,9024.0,NaN,NaN
5022,2011-02-15,8916.0,8916.0,NaN,NaN


In [145]:
# # only the data from 2008-2011 has this issue

# # Drop rows where both 'open' and 'high' are null
# df = df.dropna(subset=['open', 'high'], how='all')

# # Step 2: Fill null values in 'low' and 'close' with values from 'open'
# df['low'] = df['low'].fillna(df['open']) # open == high
# df['close'] = df['close'].fillna(df['open']) # open == high

# # Display the modified DataFrame
# df

In [121]:
# Check for any missing values
print(df.isnull().sum())

date                0
open                0
high                0
low                 0
close               0
                   ..
prev_close          0
daily_range         0
gap_change          0
close_ma5_ratio     0
close_ma10_ratio    0
Length: 21, dtype: int64


In [ ]:
# Index(['date', 'open', 'high', 'low', 'close', 'gap_day', 'next_close',
#        'open_change', 'high_change', 'low_change', 'close_change',
#        'ma_close_5', 'ma_close_10', 'prev_open', 'prev_high', 'prev_low',
#        'prev_close', 'daily_range', 'gap_change', 'close_ma5_ratio',
#        'close_ma10_ratio'],
#       dtype='object')

In [155]:
# Ensure 'date' is consistently used as a datetime format
df['date'] = pd.to_datetime(df['date'])

# Features Engineering

# Create the next_close column as the target variable to be predicted
df['next_close'] = df['close'].shift(-1)

# Count the next day gap
df['gap_day'] = df['date'].shift(-1) - df['date']
# Convert gap_day to just the number of days and make it an integer
df['gap_day'] = df['gap_day'].dt.days.fillna(0).astype(int)

# Calculate the changes from the previous row for OHLC
df['open_change'] = df['open'].diff()
df['high_change'] = df['high'].diff()
df['low_change'] = df['low'].diff()
df['close_change'] = df['close'].diff()

# Function to calculate moving average over a custom date range
def custom_moving_average(df, close_col, date_col, days):
    averages = []
    for index, row in df.iterrows():
        current_date = row[date_col]
        # Get the date range
        start_date = current_date - pd.Timedelta(days=days)
        # Filter the DataFrame for the desired date range
        mask = (df[date_col] >= start_date) & (df[date_col] <= current_date)
        # Calculate the mean for the filtered values
        avg = df.loc[mask, close_col].mean()
        averages.append(avg)
    return averages

# Calculate the custom moving averages
df['ma_close_5'] = custom_moving_average(df, 'close', 'date', days=4)  # 4 previous days + today
df['ma_close_10'] = custom_moving_average(df, 'close', 'date', days=9)  # 9 previous days + today

# Capture previous values
df['prev_open'] = df['open'].shift(1)
df['prev_high'] = df['high'].shift(1)
df['prev_low'] = df['low'].shift(1)
df['prev_close'] = df['close'].shift(1)

# Add daily range (high-low)
df['daily_range'] = df['high'] - df['low']

# Add gap change (today's open - previous close)
df['gap_change'] = df['open'] - df['prev_close']

# Add close-to-moving average ratios
df['close_ma5_ratio'] = df['close'] / df['ma_close_5']
df['close_ma10_ratio'] = df['close'] / df['ma_close_10']

# Drop columns that are not needed for prediction
df = df[['date', 'open', 'high', 'low', 'close', 'gap_day', 
         'next_close', 'open_change', 'high_change', 
         'low_change', 'close_change', 'ma_close_5', 
         'ma_close_10', 'prev_open', 'prev_high', 
         'prev_low', 'prev_close', 'daily_range', 'gap_change',
         'close_ma5_ratio', 'close_ma10_ratio']]

# Filter the DataFrame to include only rows from the year 2012 onwards
df = df[df['date'] >= '2012-01-01']

# Dropping last row
df = df.drop(df.index[-1])

df

,date,open,high,low,close,gap_day,next_close,open_change,high_change,low_change,...,ma_close_5,ma_close_10,prev_open,prev_high,prev_low,prev_close,daily_range,gap_change,close_ma5_ratio,close_ma10_ratio
5249,2012-01-02,9150.0,9200.0,9067.0,9080.0,1,9135.0,50.0,-8.0,4.0,...,9074.000000,9070.833333,9100.0,9208.0,9063.0,9067.0,133.0,83.0,1.000661,1.001011
5250,2012-01-03,9090.0,9185.0,9090.0,9135.0,1,9137.0,-60.0,-15.0,23.0,...,9094.000000,9080.000000,9150.0,9200.0,9067.0,9080.0,95.0,10.0,1.004508,1.006057
5251,2012-01-04,9160.0,9185.0,9137.0,9137.0,1,9193.0,70.0,0.0,47.0,...,9117.333333,9087.125000,9090.0,9185.0,9090.0,9135.0,48.0,25.0,1.002157,1.005489
5252,2012-01-05,9175.0,9203.0,9112.0,9193.0,1,9095.0,15.0,18.0,-25.0,...,9136.250000,9104.375000,9160.0,9185.0,9137.0,9137.0,91.0,38.0,1.006212,1.009734
5253,2012-01-06,9178.0,9213.0,9095.0,9095.0,3,9150.0,3.0,10.0,-17.0,...,9128.000000,9106.875000,9175.0,9203.0,9112.0,9193.0,118.0,-15.0,0.996385,0.998696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8378,2024-10-21,15463.0,15498.0,15431.0,15495.0,1,15560.0,-60.0,-25.0,-16.0,...,15485.000000,15518.000000,15523.0,15523.0,15447.0,15465.0,67.0,-2.0,1.000646,0.998518
8379,2024-10-22,15534.0,15572.0,15534.0,15560.0,1,15620.0,71.0,74.0,103.0,...,15506.666667,15524.000000,15463.0,15498.0,15431.0,15495.0,38.0,39.0,1.003439,1.002319
8380,2024-10-23,15590.0,15633.0,15590.0,15620.0,1,15584.0,56.0,61.0,56.0,...,15558.333333,15536.000000,15534.0,15572.0,15534.0,15560.0,43.0,30.0,1.003964,1.005407
8381,2024-10-24,15640.0,15644.0,15580.0,15584.0,1,15640.0,50.0,11.0,-10.0,...,15564.750000,15539.000000,15590.0,15633.0,15590.0,15620.0,64.0,20.0,1.001237,1.002896


In [157]:
# # Count the frequency of each unique value in the 'gap_day' column
# gap_day_frequency = df['gap_day'].value_counts()

# # Display the frequency counts
# print(gap_day_frequency)

In [ ]:
# # Specify the filename
# filename = 'bloomberg_usd_idr_2012_2024_processed.csv'

# # Write the DataFrame to a CSV file
# df.to_csv(filename, index=False)

# print(f'DataFrame written to {filename}')